## Communication Graph

This notebook is dedicated to exploration and pattern finding in my cell phone bills which are in PDF formats. The ultimate goal is to make a graph out of it.

After developing a pattern, I'll make a function or a class to do everything for me.

#### Exploration and Pattern Finding

The first section is just exploration.

In [1]:
# Set up.
%matplotlib inline

import matplotlib as plt
import numpy as np
import os
import pandas as pd
import PyPDF2
import re
import seaborn
import sys

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from src.tmobile_bill_parser import (parse_bill, parse_multiple_bills)

In [2]:
bill_directory = parse_multiple_bills('bills')

ValueError: Not a valid file path.

### Introduction

- [x] Make all data the proper datatype.
- [n/a] Separate destination into city and state.
- [x] Numbers must be in roughly the same format.
- [x] Treat Text, Data, and Talk as separate tables or graphs.

In [ ]:
text_dfs = [pd.DataFrame(bill_directory[bill_period]['Text']) for bill_period in bill_directory]
data_dfs = [pd.DataFrame(bill_directory[bill_period]['Data']) for bill_period in bill_directory]
talk_dfs = [pd.DataFrame(bill_directory[bill_period]['Talk']) for bill_period in bill_directory]

text_df = pd.concat(text_dfs).reset_index()
data_df = pd.concat(data_dfs).reset_index()
talk_df = pd.concat(talk_dfs).reset_index()

In [ ]:
text_df['Amount'].value_counts()
text = text_df.drop(['Amount'], axis=1)

In [ ]:
data_df.info()
data_df.head()
data = data_df.drop(['Amount', 'Origin', 'Type', 'Service'], axis=1)

In [ ]:
talk_df['Amount'].value_counts()
talk = talk_df.drop(['Amount', 'Type'], axis=1)

In [ ]:
talk.head()
text.head()
data.head()

In [ ]:
data['Date and time'] = pd.to_datetime(data['Date and time'])
text['Date and time'] = pd.to_datetime(text['Date and time'])
talk['Date and time'] = pd.to_datetime(talk['Date and time'])
data['MB'] = pd.to_numeric(data['MB'])
talk['Min'] = pd.to_numeric(talk['Min'])

I think the Data column is good for a seeing usage over a period of time, maybe seeing if there's a pattern in the my activity over the course of a day, or days of the week I'm more active. Otherwise, I may cache that for later.

I think there's a number of graphs to be made form the Text and Talk sets.

#### Text 

- A graph between me and identifiable phone numbers, outgoing.
- A graph between me and identifiable phone numbers, incoming.
- A graph between me (Seattle) and destinations, though this may not be accurate since the destination seems to be based on the area code of the phone number.
- Activity over a day, week, or month.

#### Talk
- A weighted graph showing calls between phone numbers (people) and time talking.
- A graph between me and identifiable phone numbers, outgoing.
- A graph between me and identifiable phone numbers, incoming.

In [ ]:
data.columns
for column in data.columns:
    try:
        data[column].value_counts()['-']
    except KeyError:
        print(0)

- [x] TODO: text['Destination'] contains 629 '-'.  
- [x] TODO: phone numbers need to be normalized.  
- [x] TODO: destination needs to be normalized.

In [ ]:
#  Worth noting that area codes will never begin with 1.
phone_number_re = re.compile(r'''1?(-|\s|\.)?(\d{3}|\(\d{3}\))(-|\s|\.)?\d{3}(-|\s|\.)?\d{4}''', re.VERBOSE)
phone_str = r'1?(-|\s|\.)?(\d{3}|\(\d{3}\))(-|\s|\.)?\d{3}(-|\s|\.)?\d{4}'
destination_re = re.compile(r'(\w), (\w)')
destination_str = r'([\w\s]+), (\w+)'

In [ ]:
text_num_bool = text['Number'].str.match(phone_str)
text_dest_bool = text['Destination'].str.match(destination_str)
talk_num_bool = talk['Number'].str.match(phone_str)

In [ ]:
dest_negatives = text[text_dest_bool == False]['Destination'].value_counts()
text_negatives = text[text_num_bool == False]['Number'].value_counts()
talk_negatives = talk[talk_num_bool == False]['Number'].value_counts()

In [ ]:
def format_phone(number):
    numb = number.string
    if numb[0] == '1':
        numb = numb[1:]
    return '({}) {}-{}'.format(numb[:3], numb[3:6], numb[6:])

In [ ]:
text_norm_numbers = text[text_num_bool == True]['Number'].str.replace(r'\d{10,11}', format_phone)
talk_norm_numbers = talk[talk_num_bool == True]['Number'].str.replace(r'\d{10,11}', format_phone)

In [ ]:
final_text = text[text_num_bool == True]
final_text = final_text[final_text['Destination'].str.match(destination_str) == True]
final_talk = talk[talk_num_bool == True]

In [ ]:
final_text['Number'] = final_text['Number'].str.replace(r'\d{10,11}', format_phone)
final_talk['Number'] = final_talk['Number'].str.replace(r'\d{10,11}', format_phone)

### Initial Analysis

#### Data:
The data information is not suitable for a graph data structure. I think it would be interesting to visualize it using bar or line graphs to measure activity over time. Some questions that could be answered are:

- Which days am I more active?
- Which times throughout the day am I more active?
- What's different about days that there is more data consumption via T-Mobile's data services vice wifi?
- How has my consumption changed over time?

It's worth noting that this doesn't measure my total internet activity. It is merely seeing my data consumption through T-Mobile services. This leads to some additional inquiries:

- Does my phone track data consumption over time?
- Does Comcast track my data consumption at home and is it available to me?

#### Talk:
The are several angles to take with this data set, and much of it will be similar to __Text__. First, the graph data structure. I'd be interested in seeing total incoming and outgoing calls between me and all other nodes over the entire time period available. It could then be split into just incoming calls and just outgoing calls. I could make a dictionary of all known or easily identifiable phone numbers and make the nodes names of contacts rather than numbers. As far as node/edge weights, there's two things to consider: call frequency and call duration. This distinction is important because duration/frequency represent different measures of significance of a given contact. For example, I may have hundreds of calls to Melissa, but we usually only talk briefly to discuss logistical stuff because we are so heavily involved in each other's lives. However, I have much longer calls to my mother because she lives far away (and loves to talk). This may be a visualization problem. 

#### Text:
All the same problems of talk apply to text with exception of call duration. In addition, we know have an opportunity to graph the nodes by location (The phone number's area code really). So we could make the graph just like __Talk__. Thinking about it further, the destination isn't useful at all. Many contacts will have no affiliation (anymore) with the area code from which they first received their phone number. I think the use in the Destination column would be to have other data available by which we could compare people's actual location to the destination logged by T-Mobile.

### Additional things I'd like to do.
Looking at __Talk__, I think it would be smart to do the following:

For the graph:

- We won't care about the date and time.
- Divide the table into incoming and outgoing.
- Group by number, sum the minutes for each number, and add a 'Frequency' (Count) column to each number.

- Make a seperate graph that ignores the 'Description' distinction.

Visualization:

- I don't think it would be productive to get too granular into the time (for now), so we can further split the data weeks and months. 
- Choose the top ten most called numbers and make histograms by week and month (total, outgoing, and incoming, each). 
- Do the same as above except with call duration ('Min').

__Text__:

- Same for __Talk__, but there will be no 'Min' sum.

__Data__:

- See above for guidance.

### Other thoughts:

- I'd like to adapt the weighted graph we made in school for this project. Two issues arise with this: - It inherits from dict, thus I'd rather have it be a composition of a dict so we don't methods available that could screw up the graph. - I need to have a better understanding of graph databases to understand how this would work. 

- This graph is going to be one node(me) with a ton of leaves(my contacts). To really make this a better product, I should think about obtaining my girlfriend's bill as well as my parents. That will really make this more interesting.

In [ ]:
final_talk.head()

In [ ]:
final_text.head()